In [13]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from torch.autograd import Variable
import torch.nn as nn

In [6]:
batch_size = 50

In [8]:
dat_trn = pd.read_csv('data/trn_all.csv').values
dat_tst = pd.read_csv('data/tst_all.csv').values

y_trn = pd.read_csv('data/tst_labs.csv').values
y_tst = pd.read_csv('data/trn_labs.csv').values

In [9]:
class NumbersDataset(Dataset):
    def __init__(self, samples, labels):
        self.samples = torch.from_numpy(samples).to(torch.float32)
        self.labels = torch.from_numpy(labels).to(torch.float32)

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx], self.labels[idx]


def trn_tst_loader(trn, y_trn, tst, y_tst, batch):
    trn_torch = NumbersDataset(trn, y_trn)
    tst_torch = NumbersDataset(tst, y_tst)

    trn_loader = DataLoader(trn_torch, batch_size=batch, shuffle=True)
    tst_loader = DataLoader(tst_torch, batch_size=batch, shuffle=True)

    return trn_loader, tst_loader

In [11]:
trn_loader, tst_loader = trn_tst_loader(dat_trn, y_trn, dat_tst, y_tst, batch_size)

In [ ]:
class RNN(nn.Module):
    '''
    Inspiration from:
    https://www.kaggle.com/code/kanncaa1/recurrent-neural-network-with-pytorch/notebook

    Module used to create a recurrent neural network. 
    '''
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, batch_size):
        super(RNN, self).__init__()

        # Number of hidden dimensions
        self.hidden_dim = hidden_dim

        # Number of hidden layers
        self.layer_dim = layer_dim

        self.input_dim = input_dim
        self.batch = batch_size

        # RNN
        self.rnn = nn.RNN(2, hidden_dim, layer_dim,
                          batch_first=True, nonlinearity='relu')

        # fc: fully connected layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = Variable(torch.zeros(self.layer_dim, self.batch, self.hidden_dim))

        # One time step
        out, hn = self.rnn(x, h0)

        # Since RNN has different structure it only uses the last
        # entrance of the sequence length for the output hence [:, -1, :]
        out = self.fc(out[:, -1, :])
        return out